In [1]:
import pandas as pd


# Get all data

In [2]:
# get "uiuc-gpa-dataset.csv" from drive
share_link = 'https://drive.google.com/file/d/1AppbJcg2pCpthkVVV3eE4xDLLgWvPoDv/view?usp=sharing'
loc = 'https://drive.google.com/uc?export=download&id=' + share_link.split('/')[-2]
print(loc)
data = pd.read_csv(loc)



https://drive.google.com/uc?export=download&id=1AppbJcg2pCpthkVVV3eE4xDLLgWvPoDv


# Process Data

In [3]:
# calculate gpa
GPA_points = {
    'A+':4,
    'A':4,
    'A-':3.67,
    'B+':3.33,
    'B':3,
    'B-':2.67,
    'C+':2.33,
    'C':2,
    'C-':1.67,
    'D+':1.33,
    'D':1,
    'D-':0.67,
    'F':0,
    'W':0
}

data['Student Count'] = data[list(GPA_points.keys())].sum(axis=1)

for grade in GPA_points:
  data[grade] = data[grade] * GPA_points[grade]

data['GPA'] = data[list(GPA_points.keys())].sum(axis=1) / data['Student Count']
data

,Year,Term,YearTerm,Subject,Number,Course Title,Sched Type,A+,A,A-,...,C,C-,D+,D,D-,F,W,Primary Instructor,Student Count,GPA
0,2021,Fall,2021-fa,AAS,100,Intro Asian American Studies,DIS,8,56,7.34,...,2,0.00,0.0,0,0.00,0,0,"Zheng, Reanne",30,3.544667
1,2021,Fall,2021-fa,AAS,100,Intro Asian American Studies,DIS,0,60,0.00,...,4,0.00,0.0,0,0.00,0,0,"Atienza, Paul Michael L",27,3.394815
2,2021,Fall,2021-fa,AAS,100,Intro Asian American Studies,OD,28,16,3.67,...,6,0.00,0.0,1,0.00,0,0,"Wang, Yu",27,3.098519
3,2021,Fall,2021-fa,AAS,100,Intro Asian American Studies,DIS,4,72,0.00,...,0,0.00,0.0,0,0.00,0,0,"Zheng, Reanne",26,3.743462
4,2021,Fall,2021-fa,AAS,100,Intro Asian American Studies,DIS,0,64,3.67,...,4,0.00,0.0,0,0.00,0,0,"Atienza, Paul Michael L",28,3.547500
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
61552,2010,Summer,2010-su,STAT,410,Statistics and Probability II,LEC,20,40,7.34,...,2,5.01,0.0,0,0.00,0,0,"Stepanov, Alexei G",32,3.083750
61553,2010,Summer,2010-su,STAT,440,Statistical Data Management,LEC,16,48,29.36,...,0,0.00,0.0,0,0.00,0,0,"Unger, David",28,3.774643
61554,2010,Summer,2010-su,TAM,212,Introductory Dynamics,LEC,0,4,11.01,...,2,1.67,0.0,2,0.00,0,0,"Morgan, William T",28,2.595714
61555,2010,Summer,2010-su,TAM,251,Introductory Solid Mechanics,LCD,4,8,7.34,...,6,3.34,0.0,0,0.67,0,0,"Ott-Monsivais, Stephanie",21,2.603333


In [4]:
# Remove extraneous columns
data = data.drop(columns=list(GPA_points.keys()))
data = data.drop(columns=["Student Count", "YearTerm"])
data

,Year,Term,Subject,Number,Course Title,Sched Type,Primary Instructor,GPA
0,2021,Fall,AAS,100,Intro Asian American Studies,DIS,"Zheng, Reanne",3.544667
1,2021,Fall,AAS,100,Intro Asian American Studies,DIS,"Atienza, Paul Michael L",3.394815
2,2021,Fall,AAS,100,Intro Asian American Studies,OD,"Wang, Yu",3.098519
3,2021,Fall,AAS,100,Intro Asian American Studies,DIS,"Zheng, Reanne",3.743462
4,2021,Fall,AAS,100,Intro Asian American Studies,DIS,"Atienza, Paul Michael L",3.547500
...,...,...,...,...,...,...,...,...
61552,2010,Summer,STAT,410,Statistics and Probability II,LEC,"Stepanov, Alexei G",3.083750
61553,2010,Summer,STAT,440,Statistical Data Management,LEC,"Unger, David",3.774643
61554,2010,Summer,TAM,212,Introductory Dynamics,LEC,"Morgan, William T",2.595714
61555,2010,Summer,TAM,251,Introductory Solid Mechanics,LCD,"Ott-Monsivais, Stephanie",2.603333


# Build and save datasets as-is

In [21]:
# debug_df = data.sample(1000)  # change to fit time limit
# debug_df.to_csv("debug_data.csv", index=False)

chunk_size = 3000
working_df = data.sample(10000)  # change to fit time limit
i = chunk_size
while(i < len(working_df)):
  temp = working_df.iloc[i-chunk_size:min(i,len(working_df))]
  temp.to_csv("working_data_" + str(i) + ".csv", index=False)
  i += chunk_size
  print(i)


6000
9000
12000


# Build and save dataset with one hot vectors

In [6]:
cols = ['Subject', 'Course Title', 'Sched Type']
data = pd.get_dummies(data)


In [22]:
debug_df = data.sample(1000)  # change to fit time limit
debug_df.to_csv("debug_data_onehot.csv", index=False)

chunk_size = 3000
working_df = data.sample(10000)  # change to fit time limit
i = chunk_size
while(i < len(working_df)):
  temp = working_df.iloc[i-chunk_size:min(i,len(working_df))]
  temp.to_csv("working_data_onehot_" + str(i) + ".csv", index=False)
  i += chunk_size
  print(i)


6000
9000
12000
